In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta, date
import time
import re
import requests
from bs4 import BeautifulSoup

In [3]:
num = []
for i in range(0,240,10):
    num.append(i)


soups = [] 
for page_num in num:
    url = requests.get(f'https://www.yelp.co.uk/search?find_desc=hotels&find_loc=Berlin%2C+Germany&ns=1&start={page_num}')
    print(url)
    soups.append( BeautifulSoup(url.content, 'html.parser') )
    time.sleep(30)


<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [500]>
<Response [502]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [4]:
final_try = []
all_try =[]
hotel_price_Level =[]
hotel_names = []
number_of_reviews=[]
location = []
type_of_hotel=[]
for soup in soups:
    main_container = soup.find_all('div',class_="container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY")
    for content in main_container:

        #Resturant name 
        htags =content.find("h3",class_="css-kagwww")
        hotel_names.append(htags.text.replace('\xa0','').split('.')[1])

        #ratings
        my_try=content.find('div',{'role':'img'})
        if my_try:
            all_try.append(my_try['aria-label'].split(' ')[0])
        else:
            all_try.append(np.NaN)

        #Price_level
        princing = content.find('p', class_='css-dzq7l1')
        price_output = princing.text
        hotel_price_Level.append(re.findall('€+',price_output))

        #Number of reviews
        rates = content.find('span', class_='reviewCount__09f24__tnBk4 css-chan6m')
        if rates:
            number_of_reviews.append(rates.text)
        else:
            number_of_reviews.append(np.nan)

        #Location
        neighbour = content.find('p', class_='css-dzq7l1')
        neighb_output= neighbour.text
        location.append(re.findall('€+(\w+)',neighb_output))

        #Resturant types
        hotel_type = content.find('p', class_='css-dzq7l1')    
        types_output = hotel_type.text
        type_of_hotel.append(types_output.split('€')[0])



In [5]:
len(type_of_hotel)

220

In [6]:
##For cleaning the price column 
final_data =pd.DataFrame({'Names':hotel_names, 'Prices':hotel_price_Level,'Neighbourhoods':location,'Number_of_reviews':number_of_reviews,'Ratings':all_try,'Type_of_hotels':type_of_hotel})
final_data=final_data.mask(final_data.applymap(str).eq('[]'))
final_data=final_data.explode(['Prices'])
final_data=final_data.explode(['Neighbourhoods'])
final_data

,Names,Prices,Neighbourhoods,Number_of_reviews,Ratings,Type_of_hotels
0,The Circus,€€,Mitte,33,4.5,Hotels
1,Soho House,€€€,Prenzlauer,85,4,Hotels
2,Hotel Axel Springer am Deutschen Herzzentrum,€€€,Wedding,2,5,B&BsHotels
3,elite city guide,NaN,NaN,NaN,NaN,HotelsMitte
4,Steps-Hotel,NaN,NaN,2,4.5,HotelsHostelsWedding
...,...,...,...,...,...,...
215,Hotel Ambassador-Berlin-Grünau,NaN,NaN,NaN,NaN,HotelsKöpenick
216,Hotel Alte Feuerwache,€€,Hohenschönhausen,3,4.5,Hotels
217,Hilton Berlin,€€€,Mitte,110,4,Hotels
218,Hotel Zur Reichspost,€€,Kreuzberg,2,1,B&BsHotels


In [7]:
final_data.to_csv('Hoteldata.csv',index=False)